Gen AI Assignment 5

Devyani Mahajan

5.12.25

# Multi-Agent Customer Service System (A2A + MCP Demo)

This notebook demonstrates an end-to-end multi-agent customer service workflow using:

- A **Router Agent** (orchestrator)
- A **Customer Data Agent** (specialist over the customer/ticket database via MCP-style tools)
- A **Support Agent** (specialist generating final responses and escalating issues)

The notebook is designed to be run from:

```bash
/Users/Devyani/msads/genai/hw5/mcp_customerservice/demo.ipynb
```

with the project root:

```bash
/Users/Devyani/msads/genai/hw5/mcp_customerservice
```

Switch this to your own project root when running

In [1]:
import os, sys

PROJECT_ROOT = "/Users/Devyani/msads/genai/hw5/mcp_customerservice"
os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Working directory:", os.getcwd())

try:
    import langgraph  # type: ignore
    from langchain_openai import ChatOpenAI  # type: ignore
    print("langgraph and langchain-openai already installed.")
except Exception:
    print("Installing langgraph and langchain-openai...")
    import subprocess, sys as _sys
    subprocess.check_call([_sys.executable, "-m", "pip", "install", "langgraph>=0.5.0", "langchain-openai>=1.1"])
    print("Installation complete.")

Working directory: /Users/Devyani/msads/genai/hw5/mcp_customerservice
langgraph and langchain-openai already installed.


### Please change the code to use your API key below

In [2]:
import os

#Insert your API key here
os.environ["OPENAI_API_KEY"] = "YOUR_KEY_HERE"

print("OpenAI API key set:", bool(os.environ.get("OPENAI_API_KEY")))

OpenAI API key set: True


## 1. Database Setup via `database_setup.py`

In [3]:
from database_setup import DatabaseSetup

db = DatabaseSetup("support.db")
db.connect()
db.create_tables()
db.create_triggers()

cur = db.conn.cursor()
cur.execute("SELECT COUNT(*) FROM customers")
count = cur.fetchone()[0]

if count == 0:
    db.insert_sample_data()
    print("Inserted sample data into customers and tickets tables.")
else:
    print(f"Database already has {count} customers, skipping sample data insert.")

db.close()
print("Database ready with tables and sample data.")

Connected to database: support.db
Tables created successfully!
Triggers created successfully!
Database already has 30 customers, skipping sample data insert.
Database connection closed.
Database ready with tables and sample data.


### (Optional) Start MCP Server

In a **separate terminal**, from the same project root, you can run:

```bash
cd /Users/Devyani/msads/genai/hw5/mcp_customerservice
python mcp_server.py
```

This will start the MCP-style HTTP server exposing `tools/list` and `tools/call`.
The LangGraph workflow below uses the same underlying tool functions directly,
so the notebook can run even if the MCP server is not started, but the server
is useful for MCP Inspector and protocol validation.


## 2. Build A2A LangGraph Workflow

In [4]:
from workflows import build_a2a_graph
from langchain_core.messages import HumanMessage, AIMessage

a2a_graph = build_a2a_graph()
print("A2A graph compiled successfully.")

A2A graph compiled successfully.


## 3. Helper: Run and Inspect a Single Scenario

In [5]:
def run_a2a_scenario(title: str, query: str):
    """Run a single scenario through the A2A graph and pretty-print the trace."""
    print("======================================================")
    print(f"SCENARIO: {title}")
    print("------------------------------------------------------")
    print(f"USER QUERY: {query}")
    print("------------------------------------------------------")

    result = a2a_graph.invoke({
        "messages": [HumanMessage(content=query)]
    })

    msgs = result["messages"]

    print("\nFULL MESSAGE TRACE:\n")
    for m in msgs:
        if isinstance(m, HumanMessage):
            role = "USER"
        elif isinstance(m, AIMessage):
            role = "AGENT"
        else:
            role = "OTHER"
        print(f"{role}: {m.content}\n")

    final_response = result.get("response")
    print("------------------------------------------------------")
    print("FINAL RESPONSE:\n")
    print(final_response)

    print("\nROUTING METADATA:")
    print(f"Intent:   {result.get('intent')}")
    print(f"Urgency:  {result.get('urgency')}")
    print(f"Route:    {result.get('route')}")
    print(f"Cust ID:  {result.get('customer_id')}")
    print("======================================================\n")

## 4. Scenario 1 – Task Allocation

In [6]:
run_a2a_scenario(
    "Task Allocation (Router → CustomerDataAgent → SupportAgent)",
    "I need help with my account, customer ID 3"
)

SCENARIO: Task Allocation (Router → CustomerDataAgent → SupportAgent)
------------------------------------------------------
USER QUERY: I need help with my account, customer ID 3
------------------------------------------------------

FULL MESSAGE TRACE:

USER: I need help with my account, customer ID 3

AGENT: [RouterAgent] intent=simple_lookup, urgency=low, customer_id=3, route=data_then_support

AGENT: [CustomerDataAgent] Fetched history for customer_id=3 (tickets=2)

AGENT: [SupportAgent] Handled simple_lookup.

------------------------------------------------------
FINAL RESPONSE:

Customer 3: Bob Johnson
Email: bob.johnson@example.com
Phone: +1-555-0103
Status: disabled

ROUTING METADATA:
Intent:   simple_lookup
Urgency:  low
Route:    data_then_support
Cust ID:  3



## 5. Scenario 2 – Negotiation / Escalation

In [7]:
run_a2a_scenario(
    "Negotiation / Escalation (Cancellation + Billing Issue)",
    "I want to cancel my subscription but I am having billing issues and was charged twice. My customer ID is 2"
)

SCENARIO: Negotiation / Escalation (Cancellation + Billing Issue)
------------------------------------------------------
USER QUERY: I want to cancel my subscription but I am having billing issues and was charged twice. My customer ID is 2
------------------------------------------------------

FULL MESSAGE TRACE:

USER: I want to cancel my subscription but I am having billing issues and was charged twice. My customer ID is 2

AGENT: [RouterAgent] intent=billing_issue, urgency=high, customer_id=2, route=data_then_support

AGENT: [CustomerDataAgent] Fetched history for customer_id=2 (tickets=9)

AGENT: [SupportAgent] Created billing ticket #54 for customer_id=2 with priority=high.

------------------------------------------------------
FINAL RESPONSE:

I am sorry about the billing issue. I have created a ticket #54 with priority high. Our billing team will review your charge and process any refund that is due.

ROUTING METADATA:
Intent:   billing_issue
Urgency:  high
Route:    data_then

## 6. Scenario 3 – Multi-Step Coordination

In [8]:
run_a2a_scenario(
    "Multi-Step Coordination (Active customers with open tickets)",
    "What is the status of all high priority tickets for active customers with open tickets?"
)

SCENARIO: Multi-Step Coordination (Active customers with open tickets)
------------------------------------------------------
USER QUERY: What is the status of all high priority tickets for active customers with open tickets?
------------------------------------------------------

FULL MESSAGE TRACE:

USER: What is the status of all high priority tickets for active customers with open tickets?

AGENT: [RouterAgent] intent=active_with_open_tickets, urgency=low, customer_id=None, route=data_then_support

AGENT: [CustomerDataAgent] Listed active customers for report (count=24)

AGENT: [SupportAgent] Generated report for active customers with high priority tickets.

------------------------------------------------------
FINAL RESPONSE:

High priority ticket status for active customers:
- Customer 1 (John Doe): 2 high-priority ticket(s)
- Customer 2 (Jane Smith): 4 high-priority ticket(s)
- Customer 7 (Edward Norton): 2 high-priority ticket(s)

ROUTING METADATA:
Intent:   active_with_open_t

## 7. Assignment Test Scenarios

In [9]:
# Simple Query
run_a2a_scenario(
    "Simple Query",
    "Get customer information for ID 5"
)

# Coordinated Query (Upgrade)
run_a2a_scenario(
    "Coordinated Query (Upgrade)",
    "I am customer ID 3 and need help upgrading my account"
)

# Complex Query
run_a2a_scenario(
    "Complex Query (Active with open tickets)",
    "Show me all active customers who have open tickets"
)

# Escalation
run_a2a_scenario(
    "Escalation (Refund, urgent billing issue)",
    "I am customer ID 2 and have been charged twice, please refund immediately!"
)

# Multi-Intent
run_a2a_scenario(
    "Multi-Intent (Update email + history)",
    "I am customer ID 4, update my email to new_email@example.com and show my ticket history"
)

SCENARIO: Simple Query
------------------------------------------------------
USER QUERY: Get customer information for ID 5
------------------------------------------------------

FULL MESSAGE TRACE:

USER: Get customer information for ID 5

AGENT: [RouterAgent] intent=simple_lookup, urgency=low, customer_id=5, route=data_then_support

AGENT: [CustomerDataAgent] Fetched history for customer_id=5 (tickets=4)

AGENT: [SupportAgent] Handled simple_lookup.

------------------------------------------------------
FINAL RESPONSE:

Customer 5: Charlie Brown
Email: charlie.brown@email.com
Phone: +1-555-0105
Status: active

ROUTING METADATA:
Intent:   simple_lookup
Urgency:  low
Route:    data_then_support
Cust ID:  5

SCENARIO: Coordinated Query (Upgrade)
------------------------------------------------------
USER QUERY: I am customer ID 3 and need help upgrading my account
------------------------------------------------------

FULL MESSAGE TRACE:

USER: I am customer ID 3 and need help upgrad

## 8. Conclusion 


I learned how to coordinate agents to work together on an MCP server. This was a productive and challenging exercise in that making sure all the agents and files were coordinated correctly made troubleshooting difficult, and I had to closely evaluate each trace to find any issues, and then determine which point and which file had caused any given issue. I also had some trouble trying to run this using Gemini and GCP credits, but ultimately switched back to using an OpenAI API as shown in class for ease of use. 

I think that this was a highly useful assignment and am hopeful I will be able to apply my knowledge and skills to similar situations in the workplace. MCP can be used in widespread applications; my team built a similar customer service workflow for a United Airlines hackathon, and I'm sure I'll continue to develop similar workflows in the future.